# Introduction

An Experimental Analysis of Social Choice Algorithms
This notebook provides a comprehensive experimental framework for exploring social choice theory, a field that studies how to aggregate individual preferences into a collective decision. At its core, the challenge lies in selecting an outcome that best satisfies the group, often when we have only limited information about each individual's preferences.

The project focuses on the distinction between cardinal utilities (the "true" numerical value an agent places on an option) and ordinal preferences (a simple ranking of options). While the former contains complete information, voting rules often rely solely on the latter. This loss of information can lead to suboptimal outcomes, which is measured by a metric called distortion.

This notebook simulates three classic voting rules:

Plurality Voting: A simple "first-past-the-post" system where each agent votes for their most preferred alternative.

Veto Voting: An elimination-style rule where each agent "vetoes" their least preferred alternative. The alternative with the fewest vetoes wins.

Borda Voting: A ranked voting system where alternatives are assigned points based on their rank in each agent's preference list. The alternative with the highest total score wins.

For each simulation, the framework generates a set of agents with randomly assigned preferences and utilities. It then calculates the social welfare for every alternative—the sum of all agents' utilities for that option. This allows us to identify the optimal social welfare, which is the highest possible total utility, and the achieved social welfare of the alternative chosen by the voting rule.

Finally, the distortion is calculated as the ratio of the optimal social welfare to the achieved social welfare, i.e, Achieved Social Welfare / Optimal Social Welfare
​	

A distortion value of 1 represents a perfectly efficient outcome, while a higher value indicates a greater loss of efficiency due to the limitations of the voting rule. By comparing the distortion of Plurality, Veto, and Borda, this notebook provides a practical and quantitative analysis of their performance.

# Import Necessary Libralies

In [1]:
import numpy as np
import random
import collections

# Plurality Algorithm Implementation

This script provides a complete experimental framework for simulating the Plurality voting rule and calculating its distortion.

It combines all the previously developed functions into a single, cohesive programme that follows the UI/UX mock-up. It takes a single set of inputs (number of agents, number of alternatives, and distribution type) and outputs the key metrics for analysis.

In [2]:
# === PART 1: Data Generation Functions ===

def generate_utilities(num_agents, num_alternatives, distribution_type):
    """
    Generates a 2D array of cardinal utility values based on a specified distribution.

    Args:
        num_agents (int): The number of agents (voters).
        num_alternatives (int): The number of alternatives.
        distribution_type (str): The type of distribution to use.
                                 'uniform': Utilities are randomly and uniformly distributed between 0 and 1.
                                 'normal': Utilities follow a normal (Gaussian) distribution.

    Returns:
        list of list: A nested list where utilities[i][j] is the utility of alternative j for agent i.
    """
    utilities = []
    
    if distribution_type not in ['uniform', 'normal']:
        print(f"Warning: Invalid distribution type '{distribution_type}'. Defaulting to 'uniform'.")
        distribution_type = 'uniform'
        
    for _ in range(num_agents):
        agent_utilities = []
        if distribution_type == 'uniform':
            agent_utilities = [random.uniform(0, 1) for _ in range(num_alternatives)]
        elif distribution_type == 'normal':
            agent_utilities = [np.random.normal(loc=0.5, scale=0.2) for _ in range(num_alternatives)]
            agent_utilities = [max(0, min(1, u)) for u in agent_utilities]

        utilities.append(agent_utilities)
        
    return utilities

def generate_random_preference_list(num_alternatives):
    """
    Generates a random permutation of alternatives to represent an agent's preferences.
    """
    alternatives = [f'A{i + 1}' for i in range(num_alternatives)]
    random.shuffle(alternatives)
    return alternatives

def generate_preferences(num_agents, num_alternatives):
    """
    Simulates preferences for a given number of agents and alternatives.
    """
    preferences = []
    for _ in range(num_agents):
        preferences.append(generate_random_preference_list(num_alternatives))
    return preferences

# === PART 2: Voting Algorithm Function ===

def plurality_voting(preferences):
    """
    Implements the Plurality voting rule to find the winning alternative.
    """
    if not preferences:
        return {"winner": None, "votes": 0, "scores": {}}

    vote_counts = collections.Counter(pref[0] for pref in preferences)
    winner = None
    max_votes = -1

    for alternative, votes in vote_counts.items():
        if votes > max_votes:
            max_votes = votes
            winner = alternative

    return {
        "winner": winner,
        "votes": max_votes,
        "scores": dict(vote_counts)
    }

# === PART 3: Distortion Calculation Function ===

def calculate_distortion(utilities, winner):
    """
    Calculates the distortion of a voting rule's outcome.
    """
    if not utilities:
        return {
            "optimal_alternative": None,
            "optimal_social_welfare": 0,
            "chosen_alternative": winner,
            "achieved_social_welfare": 0,
            "distortion": 0
        }

    num_agents = len(utilities)
    num_alternatives = len(utilities[0])
    
    alternative_names = [f'A{i + 1}' for i in range(num_alternatives)]
    alt_to_index = {name: i for i, name in enumerate(alternative_names)}
    
    # 1. Calculate social welfare for all alternatives
    social_welfare = [0] * num_alternatives
    for j in range(num_alternatives):
        social_welfare[j] = sum(utilities[i][j] for i in range(num_agents))
        
    # 2. Find optimal alternative and optimal social welfare
    optimal_social_welfare = max(social_welfare)
    optimal_index = social_welfare.index(optimal_social_welfare)
    optimal_alternative = alternative_names[optimal_index]
    
    # 3. Find achieved social welfare of the winning alternative
    achieved_social_welfare = 0
    if winner in alt_to_index:
        winner_index = alt_to_index[winner]
        achieved_social_welfare = social_welfare[winner_index]

    # 4. Calculate distortion
    if achieved_social_welfare > 0:
        distortion = optimal_social_welfare / achieved_social_welfare
    else:
        distortion = float('inf')

    return {
        "optimal_alternative": optimal_alternative,
        "optimal_social_welfare": optimal_social_welfare,
        "chosen_alternative": winner,
        "achieved_social_welfare": achieved_social_welfare,
        "distortion": distortion
    }

#Run it

if __name__ == "__main__":
    num_agents = int(input("Please enter the number of agents here"))
    num_alternatives = int(input("Please enter the number of alternatives here"))
    distribution_type = str.lower(input("Please select type of distribution either 'uniform or 'normal'."))
    
    print("=== Social Choice Analysis: Plurality Voting ===")
    print(f"Parameters:")
    print(f"  - Number of Agents: {num_agents}")
    print(f"  - Number of Alternatives: {num_alternatives}")
    print(f"  - Distribution Type: {distribution_type}")
    print(f"  - Voting Rule: Plurality\n")
    
    # Generate the two crucial datasets
    cardinal_utilities = generate_utilities(num_agents, num_alternatives, distribution_type)
    ordinal_preferences = generate_preferences(num_agents, num_alternatives)
    
    # Run the voting algorithm to get the "Chosen Alternative"
    plurality_results = plurality_voting(ordinal_preferences)
    plurality_winner = plurality_results['winner']
    
    # Run the distortion calculation to get the final outputs
    distortion_results = calculate_distortion(cardinal_utilities, plurality_winner)
    
    # --- UI/UX Mock-up: Output Panel ---
    print("=== Analysis Results ===")
    print(f"  - Optimal Alternative: {distortion_results['optimal_alternative']}")
    print(f"  - Optimal Social Welfare: {distortion_results['optimal_social_welfare']:.2f}")
    print(f"  - Chosen Alternative (by Plurality): {distortion_results['chosen_alternative']}")
    print(f"  - Achieved Social Welfare: {distortion_results['achieved_social_welfare']:.2f}")
    print(f"  - Distortion: {distortion_results['distortion']:.2f}")



=== Social Choice Analysis: Plurality Voting ===
Parameters:
  - Number of Agents: 10
  - Number of Alternatives: 5
  - Distribution Type: uniform
  - Voting Rule: Plurality

=== Analysis Results ===
  - Optimal Alternative: A1
  - Optimal Social Welfare: 5.20
  - Chosen Alternative (by Plurality): A5
  - Achieved Social Welfare: 3.52
  - Distortion: 1.48


# Veto Algorithm Implementation

This script provides a complete experimental framework for simulating the Veto voting rule and calculating its distortion.

It combines all the previously developed functions into a single, cohesive programme that follows the UI/UX mock-up. It takes a single set of inputs (number of agents, number of alternatives, and distribution type) and outputs the five key metrics for analysis.

In [3]:
# === PART 1: Data Generation Functions ===

def generate_utilities(num_agents, num_alternatives, distribution_type='uniform'):
    """
    Generates a 2D array of cardinal utility values based on a specified distribution.

    Args:
        num_agents (int): The number of agents (voters).
        num_alternatives (int): The number of alternatives.
        distribution_type (str): The type of distribution to use.
                                 'uniform': Utilities are randomly and uniformly distributed between 0 and 1.
                                 'normal': Utilities follow a normal (Gaussian) distribution.

    Returns:
        list of list: A nested list where utilities[i][j] is the utility of alternative j for agent i.
    """
    utilities = []
    
    if distribution_type not in ['uniform', 'normal']:
        print(f"Warning: Invalid distribution type '{distribution_type}'. Defaulting to 'uniform'.")
        distribution_type = 'uniform'
        
    for _ in range(num_agents):
        agent_utilities = []
        if distribution_type == 'uniform':
            agent_utilities = [random.uniform(0, 1) for _ in range(num_alternatives)]
        elif distribution_type == 'normal':
            agent_utilities = [np.random.normal(loc=0.5, scale=0.2) for _ in range(num_alternatives)]
            agent_utilities = [max(0, min(1, u)) for u in agent_utilities]

        utilities.append(agent_utilities)
        
    return utilities

def generate_random_preference_list(num_alternatives):
    """
    Generates a random permutation of alternatives to represent an agent's preferences.
    """
    alternatives = [f'A{i + 1}' for i in range(num_alternatives)]
    random.shuffle(alternatives)
    return alternatives

def generate_preferences(num_agents, num_alternatives):
    """
    Simulates preferences for a given number of agents and alternatives.
    """
    preferences = []
    for _ in range(num_agents):
        preferences.append(generate_random_preference_list(num_alternatives))
    return preferences

# === PART 2: Voting Algorithm Function ===

def veto_voting(preferences):
    """
    Implements the Veto voting rule to find the winning alternative.
    Veto works by giving one negative point to the last-ranked alternative of each agent.
    The alternative with the highest total score (fewest vetoes) wins.

    Args:
        preferences (list): A 2D list of agent preferences.

    Returns:
        dict: A dictionary containing the winner, their score, and all scores.
    """
    if not preferences or not preferences[0]:
        return {"winner": None, "score": 0, "scores": {}}

    num_alternatives = len(preferences[0])
    veto_scores = {alt: 0 for alt in preferences[0]}

    for preference_list in preferences:
        last_choice = preference_list[num_alternatives - 1]
        veto_scores[last_choice] += 1
    
    winner = None
    min_vetoes = float('inf')

    for alternative, score in veto_scores.items():
        if score < min_vetoes:
            min_vetoes = score
            winner = alternative
    
    scores_for_display = {alt: len(preferences) - score for alt, score in veto_scores.items()}

    return {
        "winner": winner,
        "score": len(preferences) - min_vetoes,
        "scores": scores_for_display
    }

# === PART 3: Distortion Calculation Function ===

def calculate_distortion(utilities, winner):
    """
    Calculates the distortion of a voting rule's outcome.
    
    The distortion is the ratio of the optimal social welfare (the best possible outcome)
    to the achieved social welfare (the outcome of the chosen alternative).

    Args:
        utilities (list): A 2D list of cardinal utility values for each agent and alternative.
                          utilities[i][j] is the utility of alternative j for agent i.
        winner (str): The name of the winning alternative as determined by a voting rule
                      (e.g., 'A3').

    Returns:
        dict: A dictionary containing the five key outputs for analysis.
    """
    if not utilities:
        return {
            "optimal_alternative": None,
            "optimal_social_welfare": 0,
            "chosen_alternative": winner,
            "achieved_social_welfare": 0,
            "distortion": 0
        }

    num_agents = len(utilities)
    num_alternatives = len(utilities[0])
    
    alternative_names = [f'A{i + 1}' for i in range(num_alternatives)]
    alt_to_index = {name: i for i, name in enumerate(alternative_names)}
    
    # 1. Calculate social welfare for all alternatives
    social_welfare = [0] * num_alternatives
    for j in range(num_alternatives):
        social_welfare[j] = sum(utilities[i][j] for i in range(num_agents))
        
    # 2. Find optimal alternative and optimal social welfare
    optimal_social_welfare = max(social_welfare)
    optimal_index = social_welfare.index(optimal_social_welfare)
    optimal_alternative = alternative_names[optimal_index]
    
    # 3. Find achieved social welfare of the winning alternative
    achieved_social_welfare = 0
    if winner in alt_to_index:
        winner_index = alt_to_index[winner]
        achieved_social_welfare = social_welfare[winner_index]

    # 4. Calculate distortion
    if achieved_social_welfare > 0:
        distortion = optimal_social_welfare / achieved_social_welfare
    else:
        distortion = float('inf')

    return {
        "optimal_alternative": optimal_alternative,
        "optimal_social_welfare": optimal_social_welfare,
        "chosen_alternative": winner,
        "achieved_social_welfare": achieved_social_welfare,
        "distortion": distortion
    }

# Run it

if __name__ == "__main__":
    num_agents = int(input("Please enter the number of agents here: "))
    num_alternatives = int(input("Please enter the number of alternatives here: "))
    distribution_type = str.lower(input("Please select type of distribution either 'uniform' or 'normal': "))
    
    print("\n=== Social Choice Analysis: Veto Voting ===")
    print(f"Parameters:")
    print(f"  - Number of Agents: {num_agents}")
    print(f"  - Number of Alternatives: {num_alternatives}")
    print(f"  - Distribution Type: {distribution_type}")
    print(f"  - Voting Rule: Veto\n")
    
    # Generate the two crucial datasets
    cardinal_utilities = generate_utilities(num_agents, num_alternatives, distribution_type)
    ordinal_preferences = generate_preferences(num_agents, num_alternatives)
    
    # Run the voting algorithm to get the "Chosen Alternative"
    veto_results = veto_voting(ordinal_preferences)
    veto_winner = veto_results['winner']
    
    # Run the distortion calculation to get the final outputs
    distortion_results = calculate_distortion(cardinal_utilities, veto_winner)
    
    # --- UI/UX Mock-up: Output Panel ---
    print("\n=== Analysis Results ===")
    print(f"  - Optimal Alternative: {distortion_results['optimal_alternative']}")
    print(f"  - Optimal Social Welfare: {distortion_results['optimal_social_welfare']:.2f}")
    print(f"  - Chosen Alternative (by Veto): {distortion_results['chosen_alternative']}")
    print(f"  - Achieved Social Welfare: {distortion_results['achieved_social_welfare']:.2f}")
    print(f"  - Distortion: {distortion_results['distortion']:.2f}")



=== Social Choice Analysis: Veto Voting ===
Parameters:
  - Number of Agents: 10
  - Number of Alternatives: 5
  - Distribution Type: normal
  - Voting Rule: Veto


=== Analysis Results ===
  - Optimal Alternative: A4
  - Optimal Social Welfare: 6.10
  - Chosen Alternative (by Veto): A3
  - Achieved Social Welfare: 5.43
  - Distortion: 1.12


# Borda Algorithm Implementation

This script provides a complete experimental framework for simulating the Borda voting rule and calculating its distortion.

It combines all the previously developed functions into a single, cohesive program that follows the UI/UX mock-up. It takes a single set of inputs (number of agents, number of alternatives, and distribution type) and outputs the five key metrics for analysis.

In [4]:
# === PART 1: Data Generation Functions ===

def generate_utilities(num_agents, num_alternatives, distribution_type='uniform'):
    """
    Generates a 2D array of cardinal utility values based on a specified distribution.

    Args:
        num_agents (int): The number of agents (voters).
        num_alternatives (int): The number of alternatives.
        distribution_type (str): The type of distribution to use.
                                 'uniform': Utilities are randomly and uniformly distributed between 0 and 1.
                                 'normal': Utilities follow a normal (Gaussian) distribution.

    Returns:
        list of list: A nested list where utilities[i][j] is the utility of alternative j for agent i.
    """
    utilities = []
    
    if distribution_type not in ['uniform', 'normal']:
        print(f"Warning: Invalid distribution type '{distribution_type}'. Defaulting to 'uniform'.")
        distribution_type = 'uniform'
        
    for _ in range(num_agents):
        agent_utilities = []
        if distribution_type == 'uniform':
            agent_utilities = [random.uniform(0, 1) for _ in range(num_alternatives)]
        elif distribution_type == 'normal':
            agent_utilities = [np.random.normal(loc=0.5, scale=0.2) for _ in range(num_alternatives)]
            agent_utilities = [max(0, min(1, u)) for u in agent_utilities]

        utilities.append(agent_utilities)
        
    return utilities

def generate_random_preference_list(num_alternatives):
    """
    Generates a random permutation of alternatives to represent an agent's preferences.
    """
    alternatives = [f'A{i + 1}' for i in range(num_alternatives)]
    random.shuffle(alternatives)
    return alternatives

def generate_preferences(num_agents, num_alternatives):
    """
    Simulates preferences for a given number of agents and alternatives.
    """
    preferences = []
    for _ in range(num_agents):
        preferences.append(generate_random_preference_list(num_alternatives))
    return preferences

# === PART 2: Voting Algorithm Function ===

def borda_voting(preferences):
    """
    Implements the Borda voting rule to find the winning alternative.

    Args:
        preferences (list): A 2D list of agent preferences.

    Returns:
        dict: A dictionary containing the winner, their score, and all scores.
    """
    if not preferences or not preferences[0]:
        return {"winner": None, "score": 0, "scores": {}}
    
    num_alternatives = len(preferences[0])
    borda_scores = collections.defaultdict(int)

    # Calculate points for each alternative based on its rank
    for preference_list in preferences:
        for i, alternative in enumerate(preference_list):
            points = num_alternatives - 1 - i
            borda_scores[alternative] += points
    
    # Find the winner: the alternative with the highest Borda score.
    winner = None
    max_score = -1

    for alternative, score in borda_scores.items():
        if score > max_score:
            max_score = score
            winner = alternative
    
    return {
        "winner": winner,
        "score": max_score,
        "scores": dict(borda_scores)
    }

# === PART 3: Distortion Calculation Function ===

def calculate_distortion(utilities, winner):
    """
    Calculates the distortion of a voting rule's outcome.
    
    The distortion is the ratio of the optimal social welfare (the best possible outcome)
    to the achieved social welfare (the outcome of the chosen alternative).

    Args:
        utilities (list): A 2D list of cardinal utility values for each agent and alternative.
                          utilities[i][j] is the utility of alternative j for agent i.
        winner (str): The name of the winning alternative as determined by a voting rule
                      (e.g., 'A3').

    Returns:
        dict: A dictionary containing the five key outputs for analysis.
    """
    if not utilities:
        return {
            "optimal_alternative": None,
            "optimal_social_welfare": 0,
            "chosen_alternative": winner,
            "achieved_social_welfare": 0,
            "distortion": 0
        }

    num_agents = len(utilities)
    num_alternatives = len(utilities[0])
    
    alternative_names = [f'A{i + 1}' for i in range(num_alternatives)]
    alt_to_index = {name: i for i, name in enumerate(alternative_names)}
    
    # 1. Calculate social welfare for all alternatives
    social_welfare = [0] * num_alternatives
    for j in range(num_alternatives):
        social_welfare[j] = sum(utilities[i][j] for i in range(num_agents))
        
    # 2. Find optimal alternative and optimal social welfare
    optimal_social_welfare = max(social_welfare)
    optimal_index = social_welfare.index(optimal_social_welfare)
    optimal_alternative = alternative_names[optimal_index]
    
    # 3. Find achieved social welfare of the winning alternative
    achieved_social_welfare = 0
    if winner in alt_to_index:
        winner_index = alt_to_index[winner]
        achieved_social_welfare = social_welfare[winner_index]

    # 4. Calculate distortion
    if achieved_social_welfare > 0:
        distortion = optimal_social_welfare / achieved_social_welfare
    else:
        distortion = float('inf')

    return {
        "optimal_alternative": optimal_alternative,
        "optimal_social_welfare": optimal_social_welfare,
        "chosen_alternative": winner,
        "achieved_social_welfare": achieved_social_welfare,
        "distortion": distortion
    }

# Run it

if __name__ == "__main__":
    num_agents = int(input("Please enter the number of agents here: "))
    num_alternatives = int(input("Please enter the number of alternatives here: "))
    distribution_type = str.lower(input("Please select type of distribution either 'uniform' or 'normal': "))
    
    print("\n=== Social Choice Analysis: Borda Voting ===")
    print(f"Parameters:")
    print(f"  - Number of Agents: {num_agents}")
    print(f"  - Number of Alternatives: {num_alternatives}")
    print(f"  - Distribution Type: {distribution_type}")
    print(f"  - Voting Rule: Borda\n")
    
    # Generate the two crucial datasets
    cardinal_utilities = generate_utilities(num_agents, num_alternatives, distribution_type)
    ordinal_preferences = generate_preferences(num_agents, num_alternatives)
    
    # Run the voting algorithm to get the "Chosen Alternative"
    borda_results = borda_voting(ordinal_preferences)
    borda_winner = borda_results['winner']
    
    # Run the distortion calculation to get the final outputs
    distortion_results = calculate_distortion(cardinal_utilities, borda_winner)
    
    # --- UI/UX Mock-up: Output Panel ---
    print("\n=== Analysis Results ===")
    print(f"  - Optimal Alternative: {distortion_results['optimal_alternative']}")
    print(f"  - Optimal Social Welfare: {distortion_results['optimal_social_welfare']:.2f}")
    print(f"  - Chosen Alternative (by Borda): {distortion_results['chosen_alternative']}")
    print(f"  - Achieved Social Welfare: {distortion_results['achieved_social_welfare']:.2f}")
    print(f"  - Distortion: {distortion_results['distortion']:.2f}")



=== Social Choice Analysis: Borda Voting ===
Parameters:
  - Number of Agents: 10
  - Number of Alternatives: 5
  - Distribution Type: uniform
  - Voting Rule: Borda


=== Analysis Results ===
  - Optimal Alternative: A3
  - Optimal Social Welfare: 6.64
  - Chosen Alternative (by Borda): A4
  - Achieved Social Welfare: 4.39
  - Distortion: 1.51
